In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
import nltk
from nltk.util import ngrams

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data = pd.read_csv("/content/drive/MyDrive/Dataset/Vashantor_CSV_Format/Train/Sylhet Train Translation.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Dataset/Vashantor_CSV_Format/Test/Sylhet Test Translation.csv")
validation_data = pd.read_csv("/content/drive/MyDrive/Dataset/Vashantor_CSV_Format/Validation/Sylhet Validation Translation.csv")

In [ ]:
train_data.head(5)

,bangla_speech,banglish_speech,sylhet_bangla_speech,sylhet_banglish_speech,region_name,english_speech
0,কেমন আছো ?,kemon acho?,ভালা আছনি?,vala achoni?,Sylhet,How are you?
1,আজকে আমার মন ভালো নেই,ajke amr mon valo nei,আইজকু আমার মন ভালা নায়,aijku amar mon vala nay,Sylhet,I'm not feeling well today
2,তুমি কি করো ?,tumi ki koro?,তুমি কিতা খরো?,tumi kita khoro?,Sylhet,what are you doing
3,এই গরমে আমার কিছু ভালো লাগে না,ei gorome amar kichu valo lage na,অউ গরমো আমার কুনতা ভালা লাগের না,ou gormo amar kunta vala lager na,Sylhet,I don't like anything this summer
4,ছেলেটি সাদা রঙয়ের একটি শার্ট পরে এসেছিল,cheleti sada ronger ekti shirt pore eshechilo,ফুয়াটায় এখটা সাদা রংগর শার্ট পিন্দিয়া আইছিল,fuyatay ekhta sada ronggor shirt pindiya aichilo,Sylhet,The boy came wearing a white shirt


In [ ]:
train_data.tail(5)

,bangla_speech,banglish_speech,sylhet_bangla_speech,sylhet_banglish_speech,region_name,english_speech
1870,কাউকে মিথ্যা বলে খুশী করার থেকে সত্যি বলে কাঁদ...,kauke mittha bole khushi korar theke sotti bol...,কাউরে মিচা বলিয়া খুশির করার চেয়ে হাচা বলে কানদ...,kaure mica boliya khushir korar cheye hacha bo...,Sylhet,It's better to be happy for telling a lie than...
1871,"একে অপরের প্রতি যতো বেশী বিশ্বাস থাকবে, তাদের ...","eke oporer proti joto beshi bisshas thakbe, ta...","একে অপরের পতি যতো বেশি বিশ্বাস থাকবো, তাদের ভ...","eke oprer poti joto beshi bisshas thakbo, tad...",Sylhet,"The more trust they have in each other, the he..."
1872,বৃষ্টি বেড়ে তাপমাত্রা কমার সুখবর দিয়েছে আবহা...,brishti bere tapmatra komar sukhobor diyeche a...,মেগ বাড়িয়া তাপমাত্রা কমার সুখবর দিয়েছে আবহাওয়া...,meg bariya tapmatra komar sukhobor diyeche abh...,Sylhet,Meteorological office has given the good news ...
1873,পুরো বর্ষা মৌসুমে হাতে গোনা কয়েকদিন মাত্র বৃষ...,puro borsha moushume hate gona koyekdin matro ...,ফুরো মেগ মৌসুমো হতে গনা কয়েকদিন মাত্র মেগ অওয়া...,furo meg mousumo hote gona koyekdin matro meg ...,Sylhet,After only a handful of days of rain in the en...
1874,খাওয়ার অনেক আগেই যদি ভাত রান্না শেষ হয়ে গিয়ে থ...,khaoyar onek agei jodi vat ranna shesh hoye gi...,খাওয়ার অনেক আগেই যদি ভাত রান্দা শেষ হয়ে অইয়া গ...,khaoyar onek agei jodi vat randa shesh hoye oi...,Sylhet,If the rice is done cooking long before eating...


In [ ]:
test_data.head(5)

,bangla_speech,banglish_speech,barishal_bangla_speech,barishal_banglish_speech,region_name,english_speech
0,তোমার আব্বু কেমন আছে?,tomar abbu kemon ache?,তোমার আব্বায় ক্যামন আছে?,tomar abbay kamon ache?,Barishal,How is your father?
1,আমার বড় বোনের আজকে মন ভালো নেই,amar boro boner ajke mon valo nei,মোর বড় বুইনের আইজগো মন ভালো নাই,mor boro buiner aijgo mon valo nai,Barishal,My elder sister is not feeling well today
2,তুমি কি আমাকে এই কাজটি করে দিতে পারবে?,tumi ki amake ei kajti kore dite parbe?,তুমি কি মোরে এই কামডা কইররা দেতে পারবা?,tumi ki more ei kamda koirra dete parba?,Barishal,Can you do this for me?
3,এই গরমে বাহিরে বের হতে ইচ্ছা করে না,ei gorome bahire ber hote iccha kore na,এরহম গরমে বায়রায় বাইর হইতে ইচ্ছা করে না,erohom gorome bayray bair hoite iccha kore na,Barishal,Don't want to go out in this summer
4,তোমার সব কথাই ভুল,tomar sob kothai vul,তোমার হগল কথাই ভুল,tomar hogol kothai vul,Barishal,Everything you say is wrong


In [ ]:
test_data.tail(5)

,bangla_speech,banglish_speech,barishal_bangla_speech,barishal_banglish_speech,region_name,english_speech
370,তুমি কি আমাকে এক গ্লাস পানি এনে দিতে পারবে ?,tumi ki amake ek glass pani ene dite parbe ?,তুমি কি মোরে এক গেলাস পানি আইননা দেতে পারবা?,tumi ki more ek glass pani ainna dete parba?,Barishal,Can you get me a glass of water?
371,আমি পারবো না পানি দিতে,ami parbo na pani dite,মুই পারমু না পানি দেতে,mui parmu na pani dete,Barishal,I can't give water
372,তোমার ভাই পড়ালেখাতে অনেক মেধাবি,tomar vai poralekhate onek medhabi,তোমার ভাই পড়াল্যাহায় বেমালা মেধাবি,tomar vai poralehay bemala medhabi,Barishal,Your brother is very talented in studies
373,আচ্ছা বলো দেখি বাংলাদেশে কয়টি জেলা?,accha bolo dekhi bangladeshe koyti jela?,আচ্ছা কও দেহি বাংলাদেশে কয়ডা জেলা,accha ko dehi bangladeshe koyda jela,Barishal,Well tell me how many districts in Bangladesh?
374,সামনের দিকে যেয়ে মেয়েটি অনেক হাসবে,samner dike jeye meyeti onek hashbe,সামনের দিক যাইয়া মাইয়াডা বেমালা হাসবে,samner dik jaiya maiyada bemala hasbe,Barishal,The girl will smile a lot going forward


In [ ]:
validation_data.head(5)

,bangla_speech,banglish_speech,barishal_bangla_speech,barishal_banglish_speech,region_name,english_speech
0,বাংলাদেশে ৬৪ টা জেলা,bangladeshe ৬৪ ta jela,বাংলাদেশে ৬৪ ডা জেলা,bangladeshe ৬৪ da jela,Barishal,64 districts in Bangladesh
1,আমরা সবাই গতকাল বাহিরে গিয়েছিলাম,amra sobai gotokal bahire giyechilam,মোরা হগলডি গ্যাছেকাইল বায়রায় গেল্লাহম,mora hogoldi gechekail bayray gellahom,Barishal,We all went out yesterday
2,তোমার কথা বলার ধরন অনেক সুন্দর,tomar kotha bolar dhoron onek sundor,তোমার কথা কওয়ার বাইল বেমালা সুন্দর,tomar kotha koyar bail bemala sundor,Barishal,Your way of speaking is very nice
3,বরিশালের মানুষ কেমন হয়?,barishaler manush kemon hoy?,বরিশালের মানু ক্যামন হয়?,borishaler manu kemon hoy?,Barishal,How are the people of Barisal?
4,খুলনা জেলা কি অনেক সুন্দর?,khulna jela ki onek sundor?,খুলনা জেলা কি বেমালা সুন্দর?,khulna jela ki bemala shundor?,Barishal,Khulna district is very beautiful?


In [ ]:
validation_data.tail(5)

,bangla_speech,banglish_speech,barishal_bangla_speech,barishal_banglish_speech,region_name,english_speech
245,তুমি যাও এখান থেকে,tumi jao ekhan theke,তুমি যাও এইহান গোনে,tumi jao eihan gone,Barishal,you go from here
246,আপনি যান এখান থেকে,apni jan ekhan theke,আমনে যান এইহান গোনে,amne jan eihan gone,Barishal,You go from here
247,তুই যা এখান থেকে,tui ja ekhan theke,তুই যা এইহান গোনে,tui ja eihan gone,Barishal,you go from here
248,আপনারা পার্কে ঘুরে বেড়ান,apnara parke ghure beran,আমনেরা পার্কে ঘুইররা বেড়ান,amnera parke ghuirra beran,Barishal,You go for a walk in the park
249,উনি চেষ্টা করেন,uni cheshta koren,হ্যায় চেষ্টা করে,hey cheshta kore,Barishal,He tries


In [ ]:
!pip install transformers torch pandas

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.5 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=0c62106c27bdfaebf252cf52edb97c9b046b5b3cd556fdd477b4627e80e86d4d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
!pip install sentencepiece

In [ ]:
!pip install transformers[sentencepiece]

In [ ]:
!transformers-cli cache clear

2024-05-20 12:19:15.739864: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-20 12:19:15.739928: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-20 12:19:15.741970: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-20 12:19:17.567796: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
usage: transformers-cli <command> [<args>]
Transformers CLI tool: error: argument {convert,download,env,run,serve,login,whoami,logout,repo,add-new-model,add-new-model-like,lfs-enable-largefiles,lfs-multipart-upload,pt-to-tf}: invalid choice:

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.3 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

In [ ]:
!pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-t_tz_vwh
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-t_tz_vwh
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6859 sha256=8a2b01062dce228678bf5646d86d021a9a1f85fbf97c64930fe77129e357525c
  Stored in directory: /tmp/pip-ephem-wheel-cache-ikhkj793/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186459 sha256=6514dfb96653c447816a60bcac3dd0b69011a59

In [ ]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
#!pip install transformers==4.10.3

In [ ]:
!pip install torch transformers

In [ ]:
#!pip install accelerate==0.20.3

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, Trainer, TrainingArguments
from normalizer import normalize
import pandas as pd
import numpy as np
import nltk
from nltk.translate.bleu_score import corpus_bleu
from sacrebleu import sentence_bleu
from rouge_score import rouge_scorer
from sklearn.model_selection import train_test_split
import os


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_data = pd.read_csv("/content/drive/MyDrive/Dataset/Vashantor_CSV_Format/Train/Sylhet Train Translation.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Dataset/Vashantor_CSV_Format/Test/Sylhet Test Translation.csv")
validation_data = pd.read_csv("/content/drive/MyDrive/Dataset/Vashantor_CSV_Format/Validation/Sylhet Validation Translation.csv")

In [ ]:
train_data.head()

,bangla_speech,banglish_speech,sylhet_bangla_speech,sylhet_banglish_speech,region_name,english_speech
0,কেমন আছো ?,kemon acho?,ভালা আছনি?,vala achoni?,Sylhet,How are you?
1,আজকে আমার মন ভালো নেই,ajke amr mon valo nei,আইজকু আমার মন ভালা নায়,aijku amar mon vala nay,Sylhet,I'm not feeling well today
2,তুমি কি করো ?,tumi ki koro?,তুমি কিতা খরো?,tumi kita khoro?,Sylhet,what are you doing
3,এই গরমে আমার কিছু ভালো লাগে না,ei gorome amar kichu valo lage na,অউ গরমো আমার কুনতা ভালা লাগের না,ou gormo amar kunta vala lager na,Sylhet,I don't like anything this summer
4,ছেলেটি সাদা রঙয়ের একটি শার্ট পরে এসেছিল,cheleti sada ronger ekti shirt pore eshechilo,ফুয়াটায় এখটা সাদা রংগর শার্ট পিন্দিয়া আইছিল,fuyatay ekhta sada ronggor shirt pindiya aichilo,Sylhet,The boy came wearing a white shirt


In [ ]:
train_data = train_data.drop(columns=['banglish_speech ','sylhet_banglish_speech ','region_name ','english_speech'],axis=1)
train_data.head()

,bangla_speech,sylhet_bangla_speech
0,কেমন আছো ?,ভালা আছনি?
1,আজকে আমার মন ভালো নেই,আইজকু আমার মন ভালা নায়
2,তুমি কি করো ?,তুমি কিতা খরো?
3,এই গরমে আমার কিছু ভালো লাগে না,অউ গরমো আমার কুনতা ভালা লাগের না
4,ছেলেটি সাদা রঙয়ের একটি শার্ট পরে এসেছিল,ফুয়াটায় এখটা সাদা রংগর শার্ট পিন্দিয়া আইছিল


In [ ]:
test_data = test_data.drop(columns=['banglish_speech ','sylhet_banglish_speech ','region_name ','english_speech'],axis=1)
test_data.head()

,bangla_speech,sylhet_bangla_speech
0,তোমার আব্বু কেমন আছে?,তোমার আব্বা বালা আছইন নি?
1,আমার বড় বোনের আজকে মন ভালো নেই,আমার বড় বইনর আইজ মন ভালা নায়
2,তুমি কি আমাকে এই কাজটি করে দিতে পারবে?,তুমি কিতা মোরে এই কাজটা করাই দিতা পারবা নি?
3,এই গরমে বাহিরে বের হতে ইচ্ছা করে না,এই গরমো বাইরে বাইর অইতে জানে কয় না
4,তোমার সব কথাই ভুল,তোমার সব মাত ভুল


In [ ]:
train_data.rename(columns={'bangla_speech ': 'input_text', 'sylhet_bangla_speech '	: 'labels'}, inplace=True)
train_data.head()

,input_text,labels
0,কেমন আছো ?,ভালা আছনি?
1,আজকে আমার মন ভালো নেই,আইজকু আমার মন ভালা নায়
2,তুমি কি করো ?,তুমি কিতা খরো?
3,এই গরমে আমার কিছু ভালো লাগে না,অউ গরমো আমার কুনতা ভালা লাগের না
4,ছেলেটি সাদা রঙয়ের একটি শার্ট পরে এসেছিল,ফুয়াটায় এখটা সাদা রংগর শার্ট পিন্দিয়া আইছিল


In [ ]:
# Rename the columns to match the expected format
test_data.rename(columns={'bangla_speech ': 'input_text', 'sylhet_bangla_speech '	: 'labels'}, inplace=True)
test_data.head()

,input_text,labels
0,তোমার আব্বু কেমন আছে?,তোমার আব্বা বালা আছইন নি?
1,আমার বড় বোনের আজকে মন ভালো নেই,আমার বড় বইনর আইজ মন ভালা নায়
2,তুমি কি আমাকে এই কাজটি করে দিতে পারবে?,তুমি কিতা মোরে এই কাজটা করাই দিতা পারবা নি?
3,এই গরমে বাহিরে বের হতে ইচ্ছা করে না,এই গরমো বাইরে বাইর অইতে জানে কয় না
4,তোমার সব কথাই ভুল,তোমার সব মাত ভুল


In [ ]:
validation_data.head()

,bangla_speech,banglish_speech,sylhet_bangla_speech,sylhet_banglish_speech,region_name,english_speech
0,বাংলাদেশে ৬৪ টা জেলা,bangladeshe ৬৪ ta jela,বাংলাদেশো ৬৪ টা জেলা,bangladesho ৬৪ ta jela,Sylhet,64 districts in Bangladesh
1,আমরা সবাই গতকাল বাহিরে গিয়েছিলাম,amra sobai gotokal bahire giyechilam,আমরা হকল গতকাইল বাহিরো গিয়েছিলাম,amra hokol gotokail bahiro giyechilam,Sylhet,We all went out yesterday
2,তোমার কথা বলার ধরন অনেক সুন্দর,tomar kotha bolar dhoron onek sundor,তোমার মাত বলার ধরন বহুত সুন্দর,tomar mat bolar dhoron bohut shundor,Sylhet,Your way of speaking is very nice
3,বরিশালের মানুষ কেমন হয়?,barishaler manush kemon hoy?,বরিশালর মানুষ কিলা অইন?,barishalor manush kila oin?,Sylhet,How are the people of Barisal?
4,খুলনা জেলা কি অনেক সুন্দর?,khulna jela ki onek sundor?,খুলনা জেলা কিতা বহুত সুন্দর?,khulna jela kita bohut shundor?,Sylhet,Khulna district is very beautiful?


In [ ]:
validation_data = validation_data.drop(columns=['banglish_speech ','sylhet_banglish_speech ','region_name ','english_speech'],axis=1)
validation_data.head()

,bangla_speech,sylhet_bangla_speech
0,বাংলাদেশে ৬৪ টা জেলা,বাংলাদেশো ৬৪ টা জেলা
1,আমরা সবাই গতকাল বাহিরে গিয়েছিলাম,আমরা হকল গতকাইল বাহিরো গিয়েছিলাম
2,তোমার কথা বলার ধরন অনেক সুন্দর,তোমার মাত বলার ধরন বহুত সুন্দর
3,বরিশালের মানুষ কেমন হয়?,বরিশালর মানুষ কিলা অইন?
4,খুলনা জেলা কি অনেক সুন্দর?,খুলনা জেলা কিতা বহুত সুন্দর?


In [ ]:
# Rename the columns to match the expected format
validation_data.rename(columns={'bangla_speech ': 'input_text', 'sylhet_bangla_speech '	: 'labels'}, inplace=True)
validation_data.head()

,input_text,labels
0,বাংলাদেশে ৬৪ টা জেলা,বাংলাদেশো ৬৪ টা জেলা
1,আমরা সবাই গতকাল বাহিরে গিয়েছিলাম,আমরা হকল গতকাইল বাহিরো গিয়েছিলাম
2,তোমার কথা বলার ধরন অনেক সুন্দর,তোমার মাত বলার ধরন বহুত সুন্দর
3,বরিশালের মানুষ কেমন হয়?,বরিশালর মানুষ কিলা অইন?
4,খুলনা জেলা কি অনেক সুন্দর?,খুলনা জেলা কিতা বহুত সুন্দর?


In [ ]:
'''import torch
from transformers import MT5ForConditionalGeneration, AutoTokenizer
#https://huggingface.co/docs/transformers/model_doc/mt5
model_name = "google/mt5-small" # The variations it has -> mt5-small: 6, mt5-base: 12,mt5-large: 24, mt5-xl: 24, mt5-xxl: 24
model = MT5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the saved model
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Movies/Chittagong_BT5_model.pt")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Movies/Chittagong_BT5_tokenizer.json")

In [ ]:
import torch
from normalizer import normalize
from transformers import MT5ForConditionalGeneration, AutoTokenizer ,DataCollatorForSeq2Seq, Trainer, TrainingArguments
import os

In [ ]:
model_name = "csebuetnlp/banglat5"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
'''
# Save the model
model.save_pretrained("/content/drive/MyDrive/movie/sylhet_model.pt")

# Save the tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/movie/sylhet_tokenizer.json")'''

'\n# Save the model\nmodel.save_pretrained("/content/drive/MyDrive/movie/sylhet_model.pt")\n\n# Save the tokenizer\ntokenizer.save_pretrained("/content/drive/MyDrive/movie/sylhet_tokenizer.json")'

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class Seq2SeqDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        """
        Args:
            data: A DataFrame containing 'input_text' and 'labels' columns.
            tokenizer: A Hugging Face tokenizer.
            max_length: Maximum sequence length.
        """
        self.input_text = data['input_text'].apply(normalize).tolist()
        self.labels = data['labels'].apply(normalize).tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.input_text)

    def __getitem__(self, idx):
        input_text = self.input_text[idx]
        label_text = self.labels[idx]

        # Tokenize the input text
        input_encodings = self.tokenizer(
            input_text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Tokenize the label text to get its 'input_ids' and 'attention_mask'
        label_encodings = self.tokenizer(
            label_text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': input_encodings['input_ids'].squeeze(),
            'attention_mask': input_encodings['attention_mask'].squeeze(),
            'labels': label_encodings['input_ids'].squeeze(),
        }


In [ ]:
# Create train , test and validation datasets
train_dataset = Seq2SeqDataset(train_data, tokenizer)
test_dataset = Seq2SeqDataset(test_data, tokenizer)
validation_dataset = Seq2SeqDataset(validation_data, tokenizer)

# Create train , test and validation dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)  #batch_size=32
test_dataloader = DataLoader(test_dataset, batch_size=16) #batch_size=32
validation_dataloader = DataLoader(validation_dataset, batch_size=16) #batch_size=32


# **For training dataset**

In [ ]:
# Assuming you have already created the train_dataloader
for data_point in train_dataloader:
    input_ids = data_point['input_ids']
    attention_mask = data_point['attention_mask']
    labels = data_point['labels']
''' print("Input IDs:", input_ids)
    print("Attention Mask:", attention_mask)
    print("Labels:", labels)
'''

' print("Input IDs:", input_ids)\n    print("Attention Mask:", attention_mask)\n    print("Labels:", labels)\n'

# **For test dataset**

In [ ]:
# Assuming you have already created the test_dataloader
for data_point in test_dataloader:
    input_ids = data_point['input_ids']
    attention_mask = data_point['attention_mask']
    labels = data_point['labels']
''' print("Input IDs:", input_ids)
    print("Attention Mask:", attention_mask)
    print("Labels:", labels)
'''

' print("Input IDs:", input_ids)\n    print("Attention Mask:", attention_mask)\n    print("Labels:", labels)\n'

# **For validation dataset**

In [ ]:
# Assuming you have already created the validation_dataloader
for data_point in validation_dataloader:
    input_ids = data_point['input_ids']
    attention_mask = data_point['attention_mask']
    labels = data_point['labels']
'''    print("Input IDs:", input_ids)
    print("Attention Mask:", attention_mask)
    print("Labels:", labels)'''

'    print("Input IDs:", input_ids)\n    print("Attention Mask:", attention_mask)\n    print("Labels:", labels)'

# **from training dataset , I have done only one example decoding**

In [ ]:
# Assuming you have already created the train_dataloader with shuffle
for data_point in train_dataloader:
    input_ids = data_point['input_ids'][0]  # Assuming you want the first example
    attention_mask = data_point['attention_mask'][0]
    labels = data_point['labels'][0]

    # Decode input_ids and labels
    input_text = tokenizer.decode(input_ids, skip_special_tokens=True)
    output_text = tokenizer.decode(labels, skip_special_tokens=True)

    print("Input Text:", input_text)
    print("Output Text:", output_text)

    # Exit the loop after processing the first example
    break


Input Text: রোগীকে হাসপাতালে ভর্তি হতে হবে
Output Text: রুগীরে হাসপাতালে ভর্তি হইতে হইবে


# **from testing dataset , I have done only one example decoding**

In [ ]:
# Assuming you have already created the test_dataloader
for data_point in test_dataloader:
    input_ids = data_point['input_ids'][0]  # Assuming you want the first example
    attention_mask = data_point['attention_mask'][0]
    labels = data_point['labels'][0]

    # Decode input_ids and labels
    input_text = tokenizer.decode(input_ids, skip_special_tokens=True)
    output_text = tokenizer.decode(labels, skip_special_tokens=True)

    print("Input Text:", input_text)
    print("Output Text:", output_text)

    # Exit the loop after processing the first example
    break


Input Text: তোমার আব্বু কেমন আছে?
Output Text: তোমার আব্বায় ক্যামন আছে?


# **from validation dataset , I have done only one example decoding**

In [ ]:
# Assuming you have already created the validation_dataloader
for data_point in validation_dataloader:
    input_ids = data_point['input_ids'][0]  # Assuming you want the first example
    attention_mask = data_point['attention_mask'][0]
    labels = data_point['labels'][0]

    # Decode input_ids and labels
    input_text = tokenizer.decode(input_ids, skip_special_tokens=True)
    output_text = tokenizer.decode(labels, skip_special_tokens=True)

    print("Input Text:", input_text)
    print("Output Text:", output_text)

    # Exit the loop after processing the first example
    break


Input Text: বাংলাদেশে ৬৪ টা জেলা
Output Text: বাংলাদেশে ৬৪ ডা জেলা


In [ ]:
# Move the model to the device (CPU or GPU)
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
#pip install transformers==4.30


In [ ]:
from transformers import TrainingArguments
from transformers import AdamW

from torch.optim import AdamW

# Create a custom optimizer using torch.optim.AdamW
custom_optimizer = AdamW(
    model.parameters(),
    lr=1e-3,  # Learning rate
    eps=1e-8,  # Epsilon value to prevent division by zero
    weight_decay=0.01,  # Weight decay (L2 regularization)
)



training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Movies',
    num_train_epochs=50,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",  # Align save strategy with evaluation strategy
    save_total_limit=1,
    save_steps=700,
    learning_rate=1e-3,
    do_train=True,
    do_eval=True,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to="none",
    load_best_model_at_end=True,
    lr_scheduler_type="cosine_with_restarts",
    weight_decay=0.01,
    logging_dir='/content/drive/MyDrive/Movies',
    logging_steps=10,  # Log every 10 steps,
)



In [ ]:
from transformers import DataCollatorForSeq2Seq

# Create a data collator for sequence-to-sequence tasks
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,  # Your Hugging Face tokenizer
    model=model,
    padding=True,
    max_length=128,
    label_pad_token_id=tokenizer.pad_token_id,
)


In [ ]:
# Define the Trainer with the custom optimizer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    optimizers=(custom_optimizer, None),  # Pass the custom optimizer here
)


# **Training start here**

In [ ]:
# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,0.714500,0.527617
1,0.658500,0.493114
2,0.608900,0.457183
4,0.518500,0.432915


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr

Epoch,Training Loss,Validation Loss
0,0.714500,0.527617
1,0.658500,0.493114
2,0.608900,0.457183
4,0.518500,0.432915
5,0.561000,0.418852
6,0.522700,0.392714
8,0.520400,0.371236
9,0.400400,0.312850
10,0.277900,0.183350
12,0.149300,0.136399


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr

SafetensorError: Error while serializing: IoError(Os { code: 28, kind: StorageFull, message: "No space left on device" })

# **Saving model and tokenizer**

In [ ]:
from transformers import AutoModelForSeq2SeqLM

# Save the model
model.save_pretrained("/content/drive/MyDrive/Movies/Sylhet_BT5_model.pt")

# Save the tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/Movies/Sylhet_BT5_tokenizer.json")


# **Loading trained model and tokenizer**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the saved model
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Movies/Barishal_model.pt")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Movies/Barishal_tokenizer.json")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [ ]:
!pip install datasets

In [ ]:
!pip install python-Levenshtein

In [ ]:
!pip install jiwer

In [ ]:
# Move the model to the device (CPU or GPU)
model.to(device)

In [ ]:
!pip install rouge-score
#https://github.com/google-research/google-research/tree/master/rouge
#https://huggingface.co/spaces/evaluate-metric/rouge [Different types of ROUGE scores]

In [ ]:
!pip install evaluate

# **Loading evaluation metrics**

In [ ]:
import torch
import Levenshtein
from evaluate import load
# Define the move_to_device function
def move_to_device(batch, device):
    if isinstance(batch, torch.Tensor):
        return batch.to(device)
    elif isinstance(batch, list):
        return [move_to_device(item, device) for item in batch]
    elif isinstance(batch, dict):
        return {key: move_to_device(value, device) for key, value in batch.items()}
    else:
        return batch  # If it's not a tensor, list, or dict, leave it as is

# Load the evaluation metric for Character Error Rate (CER) and Word Error Rate (WER) and Exact Match(em)
cer_metric = load("cer")
wer_metric = load("wer")
meteor = load('meteor')
exact_match_metric = load("exact_match")

# Load BLEU and ROUGE metrics
bleu_metric = load("bleu")
rouge_metric = load('rouge')

# Initialize lists to store generated translations and references
generated_translations = []
references = []

# Generate translations for the test dataset
for batch in test_dataloader:
    # Move the batch to CUDA
    batch = move_to_device(batch, 'cuda')

    input_text = batch['input_ids']  # Access the input_text using the correct key
    labels = batch['labels']  # Access the labels using the correct key

    # Generate translations
    translation_ids = model.generate(input_text, max_length=512, num_beams=4, length_penalty=2.0, early_stopping=True)

    # Move the translation_ids to CPU to decode
    translation_ids = translation_ids.to('cpu')

    generated_translation = tokenizer.batch_decode(translation_ids, skip_special_tokens=True)

    generated_translations.extend(generated_translation)
    references.extend(tokenizer.batch_decode(labels, skip_special_tokens=True))  # Decoding the label IDs

# Make sure to move generated_translations back to CPU for evaluation if necessary
generated_translations = [translation if not isinstance(translation, str) else translation for translation in generated_translations]



In [ ]:
print("Number of generated translations:", len(generated_translations))
print("Number of references:", len(references))

Number of generated translations: 375
Number of references: 375


In [ ]:
print(generated_translations)

['তোমার আব্বা আব্বায় কিডা আব্বা কি আব্বা আব্বা আব্বা আব্বা আব্বা আব্বার করছে', 'মোর ভাই বেমালা ভালো বেমালা বেমালা ভালো লাগছে', 'তুমি এই এইডা এই কামে করতে পারবে??', 'এই লইয়া যাইতেয়ায়াছে', 'তোমার এউক্কা হ্যার লইয়া ভুল', 'তুমি মোই এউক্কা খারাপ করি', 'বিয়ার লইজ্ঞা মুই মায় মোর বাড়ি', 'মুই এউক্কা লগে লগে কইর লগে মোর লগে দ্যাখতে', 'পোলা এউক্কা ভালো লাগে না', 'মাইয়ার শাড়ি শাড়িতে শাড়ি', 'মোর দুদিন পর বিয়া পর পর বিয়া বিয়া বিয়া হইছে', 'এউক্কা এউক্কা পোলাডা এউক্কা এউক্কা এউক্কা এউক্কা এউক্কা এউক্কা এউক্কা এউক্কা এউক্কা এউক্কা এউক্কা পোলাডা এউক্কা এউক্কা পোলাডা এউক্কা এউক্কা এউক্কা আইছে', 'মাইয়ারা মইধ্যে মইধ্যে ঘুমায় না', 'মোর বাপ বিদেশ হইতে গ্যাছে', 'মোর লগে বেবাক ভালো মানু', 'মোর জামাইর লগে যায়', 'মোর ঘুরাইয়া ভালো লাগতে', 'মুই বিদেশ কইররা বেমালা বিদেশ পড়তে চাই', 'তুমি মুই তোমার মোরে খুশি না', 'হ্যায় বেবাক করে অপমান মোরে অপমান', 'মোর পোরে বইল্যাতে বেমালা বেমালা ম্যালা ভালো লাগছে', 'মোর ভাইর ছোহর বড় ভাইররা ছোমালা চায় না', 'মোর ভাইর ভাই ভাই ভাইর ভাইগো ভাই যাইতে যাইতে যাইতে চা

In [ ]:
print(references)

['তোমার আব্বায় ক্যামন আছে?', 'মোর বড় বুইনের আইজগো মন ভালো নাই', 'তুমি কি মোরে এই কামডা কইররা দেতে পারবা?', 'এরহম গরমে বায়রায় বাইর হইতে ইচ্ছা করে না', 'তোমার হগল কথাই ভুল', 'তোমার নাহান খারাপ পোলা মুই আর এউক্কাও দেহিনাই', 'বিয়ার লইজ্ঞা মায় মোর লইজ্ঞা পোলা খোজতেয়াছে', 'আব্বায় মোরে গ্যাছেকাইল এউক্কা পোলার লগে দ্যাহা করতে কইছেলে', 'পোলাডারে মোর মোডেও ভালো লাগেনাই', 'মাইয়াডা গোলাপি রঙের কাপুড় পইররা আইল্লেহ', 'মোর দুজ্ঞাদিন পরে বিয়া হইবে', 'পোলাডা এহন আর বিকালে খ্যালতে বাইর হয় না', 'মাইয়াডা বেইন্নাহালে ঘুমে গোনে উইঠ্যাই পড়তে বয়', 'মোর চাচায় বিদ্যাশ গোনে গ্যাছেকাইল বাংলাদেশে আইছে', 'মোর বাড়ির চহিদার বেমালা ভালো মানু', 'মোর দোস্ত খুলনা ঘোরতে গ্যাছে', 'মোর ঘোরতে বেমালা ভালো লাগে', 'মুই বিদ্যাশ যাইয়া বেমালা ঘোরতে চাই', 'তুমি মোরে বিরক্ত কইররো না', 'হ্যায় মোরে ম্যালা অপমান করছে', 'মোর বেইন্নাহালে উইঠ্যা বই পড়তে বেমালা ভালো লাগে', 'মোর ছোডো ভাই এক্কালেই পড়াল্যাহা করতে চায় না', 'মোর ছোডো ভাই ইস্কুলে যাইতে চায় না', 'মোর ছোডো ভাইয়ের ইস্কুলে যাইয়া পড়াল্যাহা করতে ভালো লাগে না'

In [ ]:
# Calculate Character Error Rate (CER) and Word Error Rate (WER)
results_CER = cer_metric.compute(predictions=generated_translations, references=references)
results_WER = wer_metric.compute(predictions=generated_translations, references=references)

# Calculate Exact Match (EM) and METEOR(M)
results_em = exact_match_metric.compute(predictions=generated_translations, references=references)
results_met = meteor.compute(predictions=generated_translations, references=references)

# Calculate Bilingual Evaluation Understudy (BLEU) and Recall-Oriented Understudy for Gisting Evaluation (ROUGE)
results_bleu = bleu_metric.compute(predictions=generated_translations, references=references)
results_rouge = rouge_metric.compute(predictions=generated_translations, references=references)


# Calculate Levenshtein Distance
levenshtein_distances = [Levenshtein.distance(generated, reference) for generated, reference in zip(generated_translations, references)]


# **Printing every evaluation metrics**

In [ ]:
print(results_CER)
print(results_WER)
print(results_em)
print(results_met)
print(results_bleu)
print(results_rouge)
print(levenshtein_distances)

In [ ]:
total_correct = 0
total_samples = len(references)

for generated, reference in zip(generated_translations, references):
    levenshtein_distance = Levenshtein.distance(generated, reference)
    max_length = max(len(generated), len(reference))
    accuracy = 1 - (levenshtein_distance / max_length)
    if accuracy >= 0.8:  # Adjust the threshold as needed
        total_correct += 1

accuracy = total_correct / total_samples
print("Accuracy:", accuracy)


# **Saving input text(Sylheti Language) , labels(normal bangla) and generated translations to csv**

In [ ]:
import pandas as pd

# Create a DataFrame to store translations
translation_df = pd.DataFrame({
    'input_text': test_data['input_text'],  # Assuming 'test_data' contains your test dataset
    'labels': references,
    'translations': generated_translations
})

# Save translations to a CSV file
# Save translations to a CSV file
translation_df.to_excel('/content/drive/MyDrive/dataset/Movies/Barishal_translations_BanglaT5.xlsx', index=False)
